In [23]:
import numpy as np
import random
import requests
import geopy
import pprint as pp
from typing import List, Tuple
from scipy.stats import beta

In [29]:
API_URL = "http://routing.openstreetmap.de/routed-{vehicle}/route/v1/{profile}/{lon1},{lat1};{lon2},{lat2}"
GEOLOCATOR = geopy.geocoders.Nominatim(user_agent="My Geocoder")

class Location:
    def __init__(self, name: str, lat: float, lon: float):
        self.name = name
        self.lat = lat
        self.lon = lon

    def __repr__(self):
        return str(self.__dict__)

    @classmethod
    def from_name(cls, name: str, **kwargs):
        coords = GEOLOCATOR.geocode(name, exactly_one=True)
        if coords:
            return cls(name=name, lat=coords.latitude, lon=coords.longitude, **kwargs)
        else:
            return None

    @classmethod
    def get_routes(cls, profile: str, lat1: float, lon1: float, lat2: float, lon2: float, overview: str = "simplified"):
        """
        Get the routes between two points using the OSRM API.
        - profile: Either "walking" or "driving"
        - lat1, lon1: Coordinates of the first location
        - lat2, lon2: Coordinates of the second location
        Returns the distance in meters.
        """
        if profile == "walking":
            vehicle = "foot"
        else:
            vehicle = "car"
        url = API_URL.format(vehicle=vehicle, profile=profile, lat1=lat1, lon1=lon1, lat2=lat2, lon2=lon2)
        print(url)
        params = {"overview": overview}
        response = requests.get(url, params=params)
        if response.status_code == 200:
            data = response.json()
            return data["routes"]
        else:
            raise Exception(f"Error: {response.status_code} - {response.text}")
    
    @classmethod
    def get_distance(cls, profile: str, lat1: float, lon1: float, lat2: float, lon2: float):
        """
        Get the distance between two points using the OSRM API.
        - profile: Either "walking" or "driving"
        - lat1, lon1: Coordinates of the first location
        - lat2, lon2: Coordinates of the second location
        Returns the distance in meters.
        """
        return cls.get_routes(profile, lat1, lon1, lat2, lon2, "false")[0]["distance"]

    @staticmethod
    def get_duration_by_distance(distance: float, speed: float):
        """
        Calculate duration by dividing distance by speed.
        - distance: in meters
        - speed: in meters per second
        Returns the duration in minutes.
        """
        duration_seconds = distance / speed
        return duration_seconds / 60.0  # convert to minutes

    def get_walking_duration_to(self, other, speed_mph: float = 2):
        # Get the distance in meters for walking
        distance = Location.get_distance("walking", self.lat, self.lon, other.lat, other.lon)
        # Convert MPH to M/S
        speed_mps = speed_mph * 0.44704
        return self.get_duration_by_distance(distance, speed_mps)

    def get_driving_duration_to(self, other, speed_mph: float = 10):
        # Get the distance in meters for driving
        distance = Location.get_distance("driving", self.lat, self.lon, other.lat, other.lon)
        # Convert MPH to M/S
        speed_mps = speed_mph * 0.44704
        return self.get_duration_by_distance(distance, speed_mps)


In [34]:
class ParkingStructure(Location):
    def __init__(self, name: str, lat: float, lon: float, index: int, exit_penalty: float = 5.0):
        super().__init__(name, lat, lon)
        self.index = index
        self.exit_penalty = exit_penalty

    def __repr__(self):
        return str(self.__dict__)

In [70]:
class QLearningParking:
    def __init__(self,
                 parking_structures: List[ParkingStructure],
                 start: Location,
                 end: Location,
                 prior: List[List[int]] = None,
                 alpha: float = 0.1,
                 gamma: float = 0.9,
                 epsilon: float= 0.1,
                 precomputed_duration: List[List[float]] = None):
        self.parking_structures = parking_structures
        self.start = start
        self.end = end
        self.alpha = alpha # Learning rate
        self.gamma = gamma # Discount factor
        self.epsilon = epsilon # Exploration rate

        # Initialize prior belief about the probability of finding parking at each structure using the Beta distribution.
        # prior_alpha represents success counts, prior_beta represents fail counts
        if prior:
            self.prior = prior
        else:
            self.prior = [[1.0, 1.0]] * len(parking_structures) # Uniform prior

        if precomputed_duration:
            self.duration_matrix = precomputed_duration
        else:
            self.duration_matrix = QLearningParking.precompute_duration(self.parking_structures)

        self.start_duration = [self.start.get_driving_duration_to(parking) for parking in self.parking_structures]
        self.end_duration = [parking.get_walking_duration_to(self.end) for parking in self.parking_structures]

        self.table = dict()


    @classmethod
    def success_prob_to_prior(cls, prob: float):
        return [int(round(prob * 100.0)), int(round((1 - prob) * 100.0))]

    @classmethod
    def precompute_duration(cls, parking_structures: List[ParkingStructure]):
        """
        Precompute the duration of time (driving) between all pairs of parking structures
        and store them in a matrix.
        """
        num_structures = len(parking_structures)
        duration_matrix = np.zeros((num_structures, num_structures))
        print("Precomputing duration of time between all pairs of parking structures.")
        for i in range(num_structures):
            for j in range(i + 1, num_structures):
                # Calculate driving durations between parking structures i and j

                duration = parking_structures[i].get_driving_duration_to(parking_structures[j])
                print(parking_structures[i].name, "->", parking_structures[j].name, ":", duration, "mins")
                duration_matrix[i][j] = duration
                duration_matrix[j][i] = duration # Mirror the values for j->i
        return duration_matrix

    def choose_action(self, state: Tuple[int], valid_actions: List[int]):
        """
        Using Epsilon-greedy, choose an action (parking structure).
        - valid_actions: List of indices of parking structures that are not full.
        - state: Current state as a list.
        """

        # Ensure the current state is in the Q-table
        if state not in self.table:
            self.table[state] = np.zeros(len(self.parking_structures))

        if random.uniform(0, 1) < self.epsilon:
            # Choose a random valid action
            action = random.choice(valid_actions)
        else:
            # Choose the action with the highest Q-value among valid actions
            valid_q_values = [self.table[state][action] for action in valid_actions]
            action = valid_actions[np.argmax(valid_q_values)]

        return action

    def get_reward(self, current: ParkingStructure, next_: ParkingStructure, found_parking: bool):
        """
        Determine reward
        """
        reward = 0.0
        if current == self.start:
            # driving duration from start to first parking tried
            reward -= self.start_duration[next_.index]
        else:
          if found_parking:
              # success, found parking
              self.prior[next_.index][0] += 1
              # walking duration to final destination from successful parking
              reward -= self.end_duration[current.index]
          else:
              # failure, did not find parking
              self.prior[next_.index][1] += 1

              # have to spend time driving to the next parking structure, exiting parking structure
              reward -= (current.exit_penalty + self.duration_matrix[current.index][next_.index])

        return reward

    def update(self, state: List[int], action: int, reward: float, next_state: List[int]):
        """
        Update Q-value table
        - state: Current state as a list (binary vector indicating visited parking structures)
        - action: The index of the action (parking structure) taken
        - reward: The reward received
        - next_state: Next state as a list
        """

        # Ensure the current and next states are in the Q-table
        if state not in self.table:
            self.table[state] = np.zeros(len(self.parking_structures))

        if next_state not in self.table:
            self.table[next_state] = np.zeros(len(self.parking_structures))

        # Get the current Q-value
        current_q = self.table[state][action]

        valid_actions = [i for i, ps in enumerate(self.parking_structures) if i not in next_state]
        # Calculate the maximum Q-value for the next state

        max_future_q = max([self.table[next_state][action] for action in valid_actions])

        # Update Q-value using the Bellman equation
        self.table[state][action] = current_q + self.alpha * (reward + self.gamma * max_future_q - current_q)

    def train(self, episodes: int = 1000):
        """
        Train the model
        """
        for episode in range(episodes):
            state = ()
            current = self.start
            while True:
                # Determine valid actions based on state
                valid_actions = [i for i, ps in enumerate(self.parking_structures) if i not in state]
                if not valid_actions:
                    # No valid actions left, didn't find parking, apply large negative reward.
                    reward = -9999
                    break

                action = self.choose_action(state, valid_actions)

                next_state = state + (action,)
                next_ = self.parking_structures[action]
                
                sample = beta.rvs(*self.prior[action])
                found_parking = sample > random.random()
                
                #print(found_parking, current, action)
                reward = self.get_reward(current, next_, found_parking)
                #print(action, self.table[state], reward)
                self.update(state, action, reward, next_state)
                #print("updated", action, self.table[state])
                state = next_state
                current = next_


In [71]:
# Prior for likelihood of finding parking at each parking structure
parking_probability_map = {
    "Tressider Student Union": 0.05,
    "Roble Field Garage" : 0.70,
    "Via Ortega Garage": 0.50,
    "Wilbur Field Garage": 0.10,
    "Stock Farm Garage": 0.10,
}

parking_exit_penalty_map = {
    "Tressider Student Union": 2.0,
    "Roble Field Garage" : 5.0,
    "Via Ortega Garage": 5.0,
    "Wilbur Field Garage": 5.0,
    "Stock Farm Garage": 5.0,
}

parking_structures = []
prior = []
for i, (name, prob) in enumerate(parking_probability_map.items()):
    prior.append(QLearningParking.success_prob_to_prior(prob))
    parking_structures.append(ParkingStructure.from_name(name, index=i, exit_penalty=parking_exit_penalty_map[name]))

start_point_name = "Galvez Street & El Camino Real"
start_point = Location.from_name(start_point_name)
end_point_name = "NVIDIA Auditorium"
end_point = Location.from_name(end_point_name)
print(start_point)
print(end_point)
print(prior)

{'name': 'Galvez Street & El Camino Real', 'lat': 37.4369847, 'lon': -122.1611878}
{'name': 'NVIDIA Auditorium', 'lat': 37.4281301, 'lon': -122.1742004}
[[5, 95], [70, 30], [50, 50], [10, 90], [10, 90]]


In [72]:
q_learn = QLearningParking(parking_structures=parking_structures,
                           start=start_point,
                           end=end_point,
                           prior=prior,
                           alpha=0.1,
                           gamma=0.9,
                           epsilon=0.1)

Precomputing duration of time between all pairs of parking structures.
http://routing.openstreetmap.de/routed-car/route/v1/driving/-122.1706597,37.4236765;-122.1765937,37.4262091
Tressider Student Union -> Roble Field Garage : 3.1410313767597233 mins
http://routing.openstreetmap.de/routed-car/route/v1/driving/-122.1706597,37.4236765;-122.17683561120828,37.428106
Tressider Student Union -> Via Ortega Garage : 3.9795096635647824 mins
http://routing.openstreetmap.de/routed-car/route/v1/driving/-122.1706597,37.4236765;-122.16409634597889,37.42308
Tressider Student Union -> Wilbur Field Garage : 3.3095472440944884 mins
http://routing.openstreetmap.de/routed-car/route/v1/driving/-122.1706597,37.4236765;-122.18208002336104,37.4316971
Tressider Student Union -> Stock Farm Garage : 7.33715103793844 mins
http://routing.openstreetmap.de/routed-car/route/v1/driving/-122.1765937,37.4262091;-122.17683561120828,37.428106
Roble Field Garage -> Via Ortega Garage : 0.8384782868050585 mins
http://routing

In [73]:
pp.pprint(q_learn.duration_matrix)

array([[ 0.        ,  3.14103138,  3.97950966,  3.30954724,  7.33715104],
       [ 3.14103138,  0.        ,  0.83847829,  6.23061322,  4.38849618],
       [ 3.97950966,  0.83847829,  0.        ,  7.06946433,  4.21960749],
       [ 3.30954724,  6.23061322,  7.06946433,  0.        , 11.89826414],
       [ 7.33715104,  4.38849618,  4.21960749, 11.89826414,  0.        ]])


In [74]:
q_learn.train(episodes=10000)

ValueError: max() arg is an empty sequence

In [60]:
print(q_learn.start_duration)
print(q_learn.end_duration)

[14.063245645430687, 10.829008589835361, 10.026321283703174, 12.042173705559533, 10.33762825101408]
[16.562649129086136, 8.77997494631353, 5.275441422094966, 23.318196731090435, 19.17986459078979]


In [61]:
print(q_learn.table)

{(): array([-44.10988877, -38.0173311 , -35.33508285, -38.85442457,
       -37.83627526]), (0,): array([  0.        , -39.56891091, -37.03859463, -36.548178  ,
       -36.04118616]), (0, 1): array([  0.        ,   0.        , -28.41022999, -29.06486454,
       -28.73070835]), (0, 1, 2): array([  0.        ,   0.        ,   0.        , -24.21485562,
       -23.53765442]), (0, 1, 2, 3): array([  0.        ,   0.        ,   0.        ,   0.        ,
       -16.40326931]), (0, 1, 2, 3, 4): array([0., 0., 0., 0., 0.]), (1,): array([-33.01169286,   0.        , -29.9738663 , -34.08636195,
       -33.04724104]), (1, 0): array([  0.        ,   0.        , -28.20204833, -28.02982521,
       -28.07550267]), (1, 0, 2): array([  0.        ,   0.        ,   0.        , -20.48296451,
       -20.77958797]), (1, 0, 2, 3): array([  0.        ,   0.        ,   0.        ,   0.        ,
       -15.77456536]), (1, 0, 2, 3, 4): array([0., 0., 0., 0., 0.]), (2,): array([-37.55345251, -28.07651212,   0.      

In [62]:

policy = {}
for state, q_values in q_learn.table.items():
    mask = np.ones_like(q_values, dtype=bool)
    for visited in state:
        mask[visited] = False

    # mask off indices of parking structures we already visited
    masked_q_values = np.where(mask, q_values, -np.inf)
    best_action = np.argmax(masked_q_values)
    if best_action in state:
        policy[state] = None
    else:
        policy[state] = int(np.argmax(masked_q_values))


In [63]:
def prune_unreachable_nodes(policy):
    """
    Prunes nodes from the policy dictionary that are not reachable
    when an agent always takes the optimal action.

    Args:
        policy (dict): A dictionary where keys are tuples representing states
                       and values are integers representing optimal actions.

    Returns:
        dict: A pruned policy dictionary containing only reachable nodes.
    """
    reachable = set()  # Set to store reachable nodes
    stack = [()]       # Start from the root node (empty sequence)

    while stack:
        current = stack.pop()
        if current in reachable:
            continue  # Skip if already visited

        reachable.add(current)
        optimal_action = policy.get(current)

        if optimal_action is not None:
            # Generate the next state by appending the optimal action
            next_state = current + (optimal_action,)
            stack.append(next_state)

    # Create a new policy dictionary with only reachable nodes
    pruned_policy = {state: action for state, action in policy.items() if state in reachable}
    return pruned_policy


pruned_policy = prune_unreachable_nodes(policy)
print(pruned_policy)

{(): 2, (2,): 1, (2, 1): 4, (2, 1, 4): 0, (2, 1, 4, 0): 3, (2, 1, 4, 0, 3): None}


In [45]:
names = list(parking_probability_map.keys())
print(names)
segments = list()
for state, action in pruned_policy.items():
    print(state, action)
    text_state = tuple([names[visited] for visited in state])
    name_state = ' -> '.join(text_state)
    print("Visited:", name_state)
    optimal_action_name = names[action] if action is not None else None
    if len(state) == 0:
        segments.append([start_point, parking_structures[action]])
    elif action is None:
        segments.append([parking_structures[state[-1]], end_point])
    else:
        segments.append([parking_structures[state[-1]], parking_structures[action]])
    print("Optimal Action:", optimal_action_name)
    print()

['Tressider Student Union', 'Roble Field Garage', 'Via Ortega Garage', 'Wilbur Field Garage', 'Stock Farm Garage']
() 2
Visited: 
Optimal Action: Via Ortega Garage

(2,) 1
Visited: Via Ortega Garage
Optimal Action: Roble Field Garage

(2, 1) 4
Visited: Via Ortega Garage -> Roble Field Garage
Optimal Action: Stock Farm Garage

(2, 1, 4) 0
Visited: Via Ortega Garage -> Roble Field Garage -> Stock Farm Garage
Optimal Action: Tressider Student Union

(2, 1, 4, 0) 3
Visited: Via Ortega Garage -> Roble Field Garage -> Stock Farm Garage -> Tressider Student Union
Optimal Action: Wilbur Field Garage

(2, 1, 4, 0, 3) None
Visited: Via Ortega Garage -> Roble Field Garage -> Stock Farm Garage -> Tressider Student Union -> Wilbur Field Garage
Optimal Action: None



In [46]:
print(segments)

[[{'name': 'Palo Alto Transit Center', 'lat': 37.4434497, 'lon': -122.16516466662787}, {'name': 'Via Ortega Garage', 'lat': 37.428106, 'lon': -122.17683561120828, 'index': 2, 'exit_penalty': 5.0}], [{'name': 'Via Ortega Garage', 'lat': 37.428106, 'lon': -122.17683561120828, 'index': 2, 'exit_penalty': 5.0}, {'name': 'Roble Field Garage', 'lat': 37.4262091, 'lon': -122.1765937, 'index': 1, 'exit_penalty': 5.0}], [{'name': 'Roble Field Garage', 'lat': 37.4262091, 'lon': -122.1765937, 'index': 1, 'exit_penalty': 5.0}, {'name': 'Stock Farm Garage', 'lat': 37.4316971, 'lon': -122.18208002336104, 'index': 4, 'exit_penalty': 5.0}], [{'name': 'Stock Farm Garage', 'lat': 37.4316971, 'lon': -122.18208002336104, 'index': 4, 'exit_penalty': 5.0}, {'name': 'Tressider Student Union', 'lat': 37.4236765, 'lon': -122.1706597, 'index': 0, 'exit_penalty': 2.0}], [{'name': 'Tressider Student Union', 'lat': 37.4236765, 'lon': -122.1706597, 'index': 0, 'exit_penalty': 2.0}, {'name': 'Wilbur Field Garage', '

In [48]:
import folium
import polyline
import copy

coords = [parking_structures[i].lat for i in range(len(parking_structures))], [parking_structures[i].lon for i in range(len(parking_structures))]
coordinates = list(zip(coords[0], coords[1]))

# Create a map centered at the first coordinate
map_center = end_point.lat, end_point.lon

mymap = folium.Map(location=map_center, zoom_start=15)
# Set a simplified map using the CartoDB positron tile layer
folium.TileLayer("CartoDB positron").add_to(mymap)



# Add markers for each coordinate
for idx, (lat, lon) in enumerate(coordinates):
    folium.Marker(location=[lat, lon], popup=parking_structures[idx].name).add_to(mymap)

folium.Marker(location=[start_point.lat, start_point.lon], popup=start_point_name, icon=folium.Icon(color="green", icon="info-sign")).add_to(mymap)
folium.Marker(location=[end_point.lat, end_point.lon], popup=end_point_name, icon=folium.Icon(color="red", icon="info-sign")).add_to(mymap)


map_stages = []
map_stages.append(copy.deepcopy(mymap))


In [49]:
map_stages[0]

In [50]:

routes = [([seg[0].lat, seg[0].lon], [seg[1].lat, seg[1].lon]) for seg in segments]


# Plot driving routes with a gradient of dotted lines based on dashArray
for i, route in enumerate(routes[:-1]):
    plot_routes = Location.get_routes(profile="driving", lat1=route[0][0], lon1=route[0][1], lat2=route[1][0], lon2=route[1][1], overview="simplified")
    
    # Decode the polyline and add it to the map with a gradient of dash patterns
    polyline_coords = polyline.decode(plot_routes[0]["geometry"])
    
    # Vary dash and gap lengths dynamically based on the segment index
    dash_pattern = f"{int(2)},{int(8)}"

    color = "green" if i % 2 == 0 else "orange"
    
    # Add the polyline with the dynamic dashArray pattern
    folium.PolyLine(polyline_coords, 
                    color=color, 
                    weight=2.5, 
                    opacity=0.8, 
                    dashArray=dash_pattern).add_to(mymap)

    temp_copy = copy.deepcopy(mymap)

    # Plot the walking route (in red, with a fixed dotted pattern)
    plot_routes = Location.get_routes(profile="walking", lat1=route[1][0], lon1=route[1][1], lat2=routes[-1][1][0], lon2=routes[-1][1][1], overview="simplified")
    polyline_coords = polyline.decode(plot_routes[0]["geometry"])
    folium.PolyLine(polyline_coords, color="red", weight=2.5, opacity=0.8, dashArray=dash_pattern).add_to(temp_copy)
    map_stages.append(temp_copy)

# Plot the walking route (in red, with a fixed dotted pattern)
plot_routes = Location.get_routes(profile="walking", lat1=routes[-1][0][0], lon1=routes[-1][0][1], lat2=routes[-1][1][0], lon2=routes[-1][1][1], overview="simplified")
polyline_coords = polyline.decode(plot_routes[0]["geometry"])
folium.PolyLine(polyline_coords, color="red", weight=2.5, opacity=0.8, dashArray=dash_pattern).add_to(mymap)

# Save the final map to an HTML file
mymap.save(f"map.html")


http://routing.openstreetmap.de/routed-car/route/v1/driving/-122.16516466662787,37.4434497;-122.17683561120828,37.428106
http://routing.openstreetmap.de/routed-foot/route/v1/walking/-122.17683561120828,37.428106;-122.1742004,37.4281301
http://routing.openstreetmap.de/routed-car/route/v1/driving/-122.17683561120828,37.428106;-122.1765937,37.4262091
http://routing.openstreetmap.de/routed-foot/route/v1/walking/-122.1765937,37.4262091;-122.1742004,37.4281301
http://routing.openstreetmap.de/routed-car/route/v1/driving/-122.1765937,37.4262091;-122.18208002336104,37.4316971
http://routing.openstreetmap.de/routed-foot/route/v1/walking/-122.18208002336104,37.4316971;-122.1742004,37.4281301
http://routing.openstreetmap.de/routed-car/route/v1/driving/-122.18208002336104,37.4316971;-122.1706597,37.4236765
http://routing.openstreetmap.de/routed-foot/route/v1/walking/-122.1706597,37.4236765;-122.1742004,37.4281301
http://routing.openstreetmap.de/routed-car/route/v1/driving/-122.1706597,37.4236765;-1

In [51]:
map_stages[1]

In [52]:
map_stages[2]

In [53]:
map_stages[3]

In [54]:
map_stages[4]

In [55]:
map_stages[5]